# Advent of Code 2025


# Puzzle - part 1

**--- Day 5: Cafeteria ---**
As the forklifts break through the wall, the Elves are delighted to\
discover that there was a cafeteria on the other side after all.

You can hear a commotion coming from the kitchen.\
"At this rate, we won't have any time left to put the wreaths up in the dining hall!"\
Resolute in your quest, you investigate.

"If only we hadn't switched to the new inventory management system right before Christmas!" another Elf exclaims.\
You ask what's going on.

The Elves in the kitchen explain the situation: because of their complicated new inventory management system,\
they can't figure out which of their ingredients are **fresh** and which are **spoiled**. When you ask how it works,\
they give you a copy of their database (your puzzle input).

The database operates on **ingredient IDs**.\
It consists of a list of **fresh ingredient ID ranges**, a blank line, and a list of **available ingredient IDs**.\
For example:
```
3-5
10-14
16-20
12-18

1
5
8
11
17
32
```

The fresh ID ranges are **inclusive**: the range `3-5` means that ingredient IDs `3`, `4`, and `5` are all **fresh**.\
The ranges can also **overlap**; an ingredient ID is fresh if it is in **any** range.

The Elves are trying to determine which of the **available ingredient IDs** are **fresh**.\
In this example, this is done as follows:

- Ingredient ID `1`is spoiled because it does not fall into any range.
- Ingredient ID `5`is **fresh** because it falls into range `3-5`.
- Ingredient ID `8`is spoiled.
- Ingredient ID `11` is **fresh** because it falls into range `10-14`.
- Ingredient ID `17` is **fresh** because it falls into range `16-20` as well as range `12-18`.
- Ingredient ID `32` is spoiled.

So, in this example, `3` of the available ingredient IDs are fresh.

Process the database file from the new inventory management system.\
**How many of the available ingredient IDs are fresh?**

## Input

In [21]:
# Load the input file

with open('input - Day 5.txt', 'r') as file:
    file_input = file.read()


print(file_input[:504])

527028483603237-530596046705806
231611686269733-232830228664181
75475032395596-76960084164171
203839219483157-205309276440608
50899806602568-58580276960637
348023446284659-349432400960533
99300079175333-99300079175333
243603050472076-244121129344428
215608453928184-218347151677708
559603756857369-560747959347574
357973561889555-361367389958275
153034818227400-159360207888955
502733798128162-503279006301491
323013740729363-323381915985523
527028483603237-530596046705806
61641386395983-67021230322014



## Input Formatting

Let's extract each row first

In [22]:
from pprint import pprint

input_sections = file_input.split('\n\n')
print(f"This input has {len(input_sections)} sections")

fresh_ranges_input = input_sections[0].split("\n")
ingredients_input = input_sections[1].split("\n")


print(f"There are {len(fresh_ranges_input)} fresh ranges")
print(f"There are {len(ingredients_input)} ingredients")

This input has 2 sections
There are 190 fresh ranges
There are 1000 ingredients


Make sure the last row is not empty

In [23]:
print(f"Last row: {fresh_ranges_input[-1]}")
# Check the last row, if it's empty then let's remove it
if fresh_ranges_input[-1] == "":
    del fresh_ranges_input[-1]

print(len(fresh_ranges_input), "\n")

print(f"Last row: {ingredients_input[-1]}")
# Check the last row, if it's empty then let's remove it
if ingredients_input[-1] == "":
    del ingredients_input[-1]

print(len(ingredients_input))

Last row: 486150733366819-488568724471762
190 

Last row: 268219264421529
1000


Let's make those ranges into tuples

In [24]:
fresh_ranges = []

for fresh_range in fresh_ranges_input:
    bounds = fresh_range.split("-")
    lower_bound, upper_bound  = int(bounds[0]), int(bounds[1])

    fresh_ranges.append((lower_bound, upper_bound))

print(len(fresh_ranges))

190


And let's make numbers into ints

In [25]:
ingredients = []

for ingredient in ingredients_input:
    ingredients.append(int(ingredient))

print(len(ingredients))

1000


## Solution

Those ranges are pretty  big, no point in expanding them.\
Instead we will check if the `ID` is in bounds.

In [26]:
def check_fresh(id, ranges):

    for lower_bound, upper_bound in ranges:
        if lower_bound <= id <= upper_bound:
            return True

    return False


In [27]:
fresh_ingredients = []

for ingredient_id in ingredients:

    if check_fresh(ingredient_id, fresh_ranges):
        fresh_ingredients.append(ingredient_id)

print(f"There are {len(fresh_ingredients)} fresh ingredients")
# 868

There are 868 fresh ingredients


# Puzzle - part 2

**--- Part Two ---**
The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.

So that they can stop bugging you when they get new inventory,\
the Elves would like to know **all** of the IDs that the **fresh ingredient ID ranges** consider to be **fresh**.\
An ingredient ID is still considered fresh if it is in any range.

Now, the second section of the database (the available ingredient IDs) is irrelevant.\
Here are the fresh ingredient ID ranges from the above example:
```
3-5
10-14
16-20
12-18
```

The ingredient IDs that these ranges consider to be fresh are\
 `3`, `4`, `5`, `10`, `11`, `12`, `13`, `14`, `15`, `16`, `17`, `18`, `19`, and `20`.\
 So, in this example, the fresh ingredient ID ranges consider a total of `14` ingredient IDs to be fresh.

Process the database file again.\
**How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?**

## Solution

This is relatively simple, once again we don't want to expand the ranges themselves.\
Instead, we simply do `upper bound - lower bound + 1` - since bounds are inclusive.\

So if we have a range `5 - 10` if we do `10 - 5 = 5` that's incorrect because in fact we have:\
`5`, `6`, `7`, `8`, `9`, `10` which makes for a total of `6` ingredient IDs.

The problem it the overlap.\
Notice that `5-10` and `7-12` will still only give us `12 - 5 + 1 = 7`.\
So we can't simply do `( 10 - 5 + 1 = 6 ) + ( 12 - 7 + 1 = 6 ) = 12`.\

First we need to merge the overlapping ranges.\
We will create a function very similar to `check_fresh` but one that\
also return the ID of the matched overlapping range.

In [28]:
def check_overlap(search_range, ranges):

    search_lower_bound, search_upper_bound = search_range

    for idx,(lower_bound, upper_bound) in enumerate(ranges):

        if lower_bound == search_lower_bound and upper_bound == search_upper_bound:
            continue

        if lower_bound <= search_lower_bound <= upper_bound:
            return lower_bound, max(search_upper_bound, upper_bound), idx

        elif lower_bound <= search_upper_bound <= upper_bound:
            return min(search_lower_bound, lower_bound), upper_bound, idx

    return False


We will iterate once, then when we find overlap stop.\
Once we are stopped we will edit the ranges to remove old two ranges and add one new one.\
We will continue with the new added range.

In [29]:
# While true we continue to update the list
# and after each update we iterate over it again
while True:

    # I know we technically don't need these
    # but they make for a clean code
    overlap_search  = None
    idxs_to_delete  = None

    new_lower_bound = None
    new_upper_bound = None

    # Iterate over the current list
    for current_idx, fresh_range in enumerate(fresh_ranges):

        # Look for overlaps
        overlap_search = check_overlap(fresh_range, fresh_ranges)

        # If no overlap continue to the next range
        if overlap_search == False:
            continue
        else:
            new_lower_bound, new_upper_bound, matched_idx = overlap_search
            idxs_to_delete = [current_idx, matched_idx]
            break

    # If we found an overlap in the loop
    if overlap_search != False:
        # Iterate backwards so that when we remove the indices
        # we don't shift the ones after it, hence reverse iteration
        idxs_to_delete.sort(reverse=True)

        for idx in idxs_to_delete:
            del fresh_ranges[idx] # remove from the current list

        # add our new range (we removed the old ones)
        fresh_ranges.append((new_lower_bound, new_upper_bound))
    else:
        # if we didn't find any overlaps we can finish
        break

# Make sure there are no duplicates (since we ignore those in check_overlap)
fresh_ranges = list(set(fresh_ranges));

Now we can do simple `upper bound - lower bound + 1`

In [30]:
fresh_ids = []

for lower_bound, upper_bound in fresh_ranges:
    fresh_ids.append(upper_bound - lower_bound + 1)

print(f"There are {sum(fresh_ids)} IDs that can be fresh")
# 354143734113772

There are 354143734113772 IDs that can be fresh
